In [1]:
import cv2
import mediapipe as mp
import math

In [2]:
mphands = mp.solutions.hands
hand = mphands.Hands()
mpdraw = mp.solutions.drawing_utils

In [3]:
video = cv2.VideoCapture(0)

def distance(p1, p2):
    """Calculate distance between two points"""
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def recognize_gesture(lmlist):
    """Recognize hand gestures from landmarks"""
    if len(lmlist) != 21:
        return "No Hand"
    
    # Extract key points
    thumb = lmlist[4]
    index = lmlist[8]
    middle = lmlist[12]
    ring = lmlist[16]
    pinky = lmlist[20]
    palm = lmlist[0]
    
    # Finger tips
    tipid = [4, 8, 12, 16, 20]
    
    # Count extended fingers
    fingers = []
    
    # Thumb (special case - left/right hand detection)
    if lmlist[2][0] < lmlist[3][0]:  # Right hand
        if lmlist[4][0] < lmlist[3][0]:
            fingers.append(1)
        else:
            fingers.append(0)
    else:  # Left hand
        if lmlist[4][0] > lmlist[3][0]:
            fingers.append(1)
        else:
            fingers.append(0)
    
    # Other four fingers
    for i in range(1, 5):
        if lmlist[tipid[i]][1] < lmlist[tipid[i] - 2][1]:
            fingers.append(1)
        else:
            fingers.append(0)
    
    # Recognize specific gestures
    finger_count = fingers.count(1)
    
    # 1. PEACE (Victory sign) - Index and Middle up, others down
    if fingers == [0, 1, 1, 0, 0]:
        return "PEACE"
    
    # 2. OK SIGN - Thumb and index together, others up
    thumb_index_dist = distance(lmlist[4][0:2], lmlist[8][0:2])
    if thumb_index_dist < 30 and fingers[2] == 1 and fingers[3] == 1 and fingers[4] == 1:
        return "OK"
    
    # 3. THUMBS UP - Only thumb up, fingers closed
    if fingers == [1, 0, 0, 0, 0] and lmlist[4][1] < lmlist[3][1]:
        return "THUMBS UP"
    
    # 4. THUMBS DOWN - Only thumb down, fingers closed
    if fingers == [1, 0, 0, 0, 0] and lmlist[4][1] > lmlist[3][1]:
        return "THUMBS DOWN"
    
    # 5. ROCK SIGN - Index and Pinky up, others down
    if fingers == [0, 1, 0, 0, 1]:
        return "ROCK"
    
    # 6. STOP SIGN - All fingers up
    if fingers == [1, 1, 1, 1, 1]:
        return "STOP/TALK TO HAND"
    
    # 7. POINTING - Only index up
    if fingers == [0, 1, 0, 0, 0]:
        return "POINTING"
    
    # 8. THREE FINGERS - Thumb, Index, Middle up
    if fingers == [1, 1, 1, 0, 0]:
        return "THREE"
    
    # 9. FOUR FINGERS - All except pinky up
    if fingers == [1, 1, 1, 1, 0]:
        return "FOUR"
    
    # 10. FIST - All fingers down
    if fingers == [0, 0, 0, 0, 0]:
        return "FIST"
    
    return "UNKNOWN"

while True:
    suc, img = video.read()
    if not suc:
        break
    
    h, w, c = img.shape
    img1 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hand.process(img1)
    
    lmlist = []
    gesture = "No Hand"
    
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            for id, lm in enumerate(hand_landmarks.landmark):
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmlist.append([lm.x, lm.y, lm.z])
            
            # Recognize gesture when we have all 21 landmarks
            if len(lmlist) == 21:
                gesture = recognize_gesture(lmlist)
            
            # Draw hand landmarks
            mpdraw.draw_landmarks(
                img, 
                hand_landmarks, 
                mphands.HAND_CONNECTIONS,
                mpdraw.DrawingSpec(color=(0, 0, 255), circle_radius=5, thickness=2),
                mpdraw.DrawingSpec(color=(0, 255, 0), thickness=2)
            )
    
    # Display gesture on screen
    cv2.putText(img, f"Gesture: {gesture}", (50, 100), 
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
    cv2.putText(img, "Press 'q' to exit", (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    cv2.imshow("Hand Gesture Recognition", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()